In [1]:
%matplotlib inline
%load_ext memory_profiler

import scanpy as sc
import numpy as np
from scipy import sparse
import os,time,datetime,sys,pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

sc.settings.verbosity=2

In [2]:
# load data
pfp = '/home/ngr4/project/scnd/results/'
pdfp = '/home/ngr4/project/scnd/data/processed/'

if True :
    start=time.time()
    fname='fullnoimp.h5ad'
    backed='r' # None if not, 'r+' if want to modify AnnData
    adata = sc.read_h5ad(os.path.join(pdfp,fname),backed=backed)
    print('Data obj loaded in {:.2f}-s @'.format(time.time()-start)+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))

Data obj loaded in 15.99-s @200226.20:53:57


## Early development prediction

Induction task with all cell types

#### TODO:
- try with just PC
- few show learning with PC (10 labels, e.g., 5wk_WT, 5wk_SCA1, etc.)

In [3]:
# split data 

## sampling for induction, early devel pred
### 5wk SCA1 [7294, 72931, 72932]
### 5wk WT [7202, 72921, 72922]
idx_train = np.arange(adata.shape[0])[(adata.obs['batch']!='7294') & (adata.obs['batch']!='7202')]
idx_test = np.arange(adata.shape[0])[(adata.obs['batch']=='7294') | (adata.obs['batch']=='7202')]

batchid_train = adata.obs['batch'][idx_train].to_list()
batchid_test = adata.obs['batch'][idx_test].to_list()

idx_train,_=train_test_split(idx_train,train_size=0.2,stratify=batchid_train)
idx_test,_=train_test_split(idx_test,train_size=0.5,stratify=batchid_test)
idx_train.sort()
idx_test.sort()

data_train = sc.AnnData(adata.X[idx_train.tolist()], obs=adata.obs.iloc[idx_train.tolist(),:]) # more mem tax, %memit sc.AnnData(sparse.csr_matrix(adata.X[idx_train.tolist()]))
data_test = sc.AnnData(adata.X[idx_test.tolist()], obs=adata.obs.iloc[idx_test.tolist(),:]) 

if True :
        scaler = MinMaxScaler()
        data_train.X = scaler.fit_transform(data_train.X)
        scaler = MinMaxScaler()
        data_test.X = scaler.fit_transform(data_test.X)
data_train.X = sparse.csr_matrix(data_train.X)
data_test.X = sparse.csr_matrix(data_test.X)

sc.pp.neighbors(data_train,n_neighbors=10,n_pcs=100)
sc.pp.neighbors(data_test,n_neighbors=10,n_pcs=100)

adj_train=data_train.uns['neighbors']['connectivities']+sparse.csr_matrix(np.eye(data_train.shape[0]))
adj_test=data_test.uns['neighbors']['connectivities']+sparse.csr_matrix(np.eye(data_test.shape[0]))



computing neighbors
         Falling back to preprocessing with `sc.pp.pca` and default params.
computing PCA with n_comps = 50
    finished (0:01:10)


/gpfs/ysm/project/dijk/ngr4/conda_envs/py37dev/lib/python3.7/site-packages/numba/typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../conda_envs/py37dev/lib/python3.7/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  state.func_ir.loc))
/gpfs/ysm/project/dijk/ngr4/conda_envs/py37dev/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../co

    finished (0:01:28)
computing neighbors
         Falling back to preprocessing with `sc.pp.pca` and default params.
computing PCA with n_comps = 50
    finished (0:00:22)


/gpfs/ysm/project/dijk/ngr4/conda_envs/py37dev/lib/python3.7/site-packages/numba/typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../conda_envs/py37dev/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  state.func_ir.loc))


    finished (0:00:24)


In [4]:
# io 
if True :
    # export
    train = {'labels':(data_train.obs['genotype']=='SCA1').astype(int).to_numpy(),
             'adj':adj_train.todense(),
             'node_features':data_train.X.todense()}
    test = {'labels':(data_test.obs['genotype']=='SCA1').astype(int).to_numpy(),
             'adj':adj_test.todense(),
             'node_features':data_test.X.todense()}
    with open(os.path.join(pdfp,'earlyDev_train.pickle'),'wb') as f:
        pickle.dump(train, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(pdfp,'earlyDev_test.pickle'),'wb') as f:
        pickle.dump(test, f, protocol=pickle.HIGHEST_PROTOCOL)
if False :
    # import
    with open(os.path.join(pdfp,'earlyDev_train_1percent.pickle'),'rb') as f:
        dd = pickle.load(f) # test if b, np.sum(b['adj']==adj_train)==adj_train.shape[0]**2

## Predict timepoint

With just WT or pooled?